In [4]:
import os
import cv2
import numpy as np
import random
from glob import glob
from rtmlib import Wholebody  

# Функция сдвига кадра
def shift_frame(frame, shift_x):
    (h, w) = frame.shape[:2]
    M = np.float32([[1, 0, shift_x], [0, 1, 0]])
    shifted = cv2.warpAffine(frame, M, (w, h), borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
    return shifted

# Функция аугментации: горизонтальное отражение или сдвиг
def augment_frames(frames):
    aug_type = random.choice(['flip', 'shift'])
    if aug_type == 'flip':
        return [cv2.flip(frame, 1) for frame in frames]
    else:
        shift_val = random.choice([-30, 30])
        return [shift_frame(frame, shift_val) for frame in frames]

# Функция сохранения фрагмента видео
def save_fragment(frames, output_path, fps, frame_size, fourcc):
    writer = cv2.VideoWriter(output_path, fourcc, fps, frame_size)
    for frame in frames:
        writer.write(frame)
    writer.release()

# Функция для вычисления угла между тремя точками
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-8)
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

# Функция обработки видео для категорий "true" и "false"
def process_videos_angle_subdir(video_dir, output_dir, segment_length=8, do_augmentation=True):
    """
    Обрабатывает видео из video_dir.
    Разбивает видео на сегменты длиной segment_length кадров.
    Для каждого сегмента вычисляются углы локтей. 
    Если средний угол обеих рук < 100° → метка "спуск",
    если > 150° → метка "подъем".
    Сегмент сохраняется в output_dir с именем, содержащим метку.
    Если do_augmentation=True, дополнительно сохраняется аугментированная версия.
    """
    # Параметры для модели Wholebody
    device = 'cuda'
    backend = 'onnxruntime'
    wholebody = Wholebody(mode='lightweight', backend=backend, device=device)
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    global_fps = None
    counter_down = 0
    counter_up = 0
    
    video_files = glob(os.path.join(video_dir, "*.*"))
    for video_path in video_files:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Не удалось открыть видео: {video_path}")
            continue
        fps = cap.get(cv2.CAP_PROP_FPS)
        if global_fps is None and fps > 0:
            global_fps = fps
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        frame_size = (frame_width, frame_height)
        
        segment_frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            segment_frames.append(frame)
            
            if len(segment_frames) == segment_length:
                # Вычисление углов для каждого кадра сегмента
                angles_right = []
                angles_left = []
                for frm in segment_frames:
                    keypoints, scores = wholebody(frm)
                    if np.array(scores).shape[0] > 0:
                        person_scores = scores[0]
                        person_keypoints = keypoints[0]
                        # Правые ключевые точки: индексы 6,8,10; Левые: 5,7,9 (пример)
                        if (person_scores[6] > 0.5 and person_scores[8] > 0.5 and person_scores[10] > 0.5):
                            rs = person_keypoints[6][:2]
                            re = person_keypoints[8][:2]
                            rw = person_keypoints[10][:2]
                            angle_r = calculate_angle(rs, re, rw)
                            angles_right.append(angle_r)
                        if (person_scores[5] > 0.5 and person_scores[7] > 0.5 and person_scores[9] > 0.5):
                            ls = person_keypoints[5][:2]
                            le = person_keypoints[7][:2]
                            lw = person_keypoints[9][:2]
                            angle_l = calculate_angle(ls, le, lw)
                            angles_left.append(angle_l)
                # Если углы обеих рук получены, усредняем их и определяем метку
                if len(angles_right) > 0 and len(angles_left) > 0:
                    avg_right = np.mean(angles_right)
                    avg_left = np.mean(angles_left)
                    label_segment = None
                    if avg_right < 130 and avg_left < 130:
                        label_segment = "спуск"
                    elif avg_right > 160 and avg_left > 160:
                        label_segment = "подъем"
                    
                    if label_segment is not None:
                        base_name = os.path.splitext(os.path.basename(video_path))[0]
                        if label_segment == "спуск":
                            output_filename = f"{base_name}_{label_segment}_frag_{counter_down:03d}.mp4"
                            counter_down += 1
                        else:
                            output_filename = f"{base_name}_{label_segment}_frag_{counter_up:03d}.mp4"
                            counter_up += 1
                        output_path = os.path.join(output_dir, output_filename)
                        save_fragment(segment_frames, output_path, global_fps if global_fps is not None else 30, frame_size, fourcc)
                        
                        if do_augmentation:
                            aug_segment = augment_frames(segment_frames)
                            if label_segment == "спуск":
                                output_filename = f"{base_name}_{label_segment}_frag_{counter_down:03d}_aug.mp4"
                                counter_down += 1
                            else:
                                output_filename = f"{base_name}_{label_segment}_frag_{counter_up:03d}_aug.mp4"
                                counter_up += 1
                            output_path = os.path.join(output_dir, output_filename)
                            save_fragment(aug_segment, output_path, global_fps if global_fps is not None else 30, frame_size, fourcc)
                segment_frames = []  # Очистка буфера
                
        cap.release()
        print(f"Обработано видео: {video_path}")
    
    print(f"Сегментов сохранено: Спуск - {counter_down}, Подъем - {counter_up}")

# Функция обработки видео для категории "man_doing_something"
# Для этой категории не требуется вычисление углов или аугментация – видео просто разбивается на сегменты
def process_videos_split_frames(video_dir, output_dir, segment_length=8):
    """
    Обрабатывает видео из video_dir для категории "man_doing_something".
    Разбивает видео на сегменты длиной segment_length кадров и сохраняет их без дополнительной обработки.
    """
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_files = glob(os.path.join(video_dir, "*.*"))
    counter = 0
    
    for video_path in video_files:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Не удалось открыть видео: {video_path}")
            continue
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0:
            fps = 30
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        frame_size = (frame_width, frame_height)
        
        segment_frames = []
        while True:
            ret, frame = cap.read()
            if not ret:
                # Если остались кадры, сохранить их как отдельный сегмент
                if len(segment_frames) > 0:
                    base_name = os.path.splitext(os.path.basename(video_path))[0]
                    output_filename = f"{base_name}_frag_{counter:03d}.mp4"
                    output_path = os.path.join(output_dir, output_filename)
                    save_fragment(segment_frames, output_path, fps, frame_size, fourcc)
                    counter += 1
                break
            segment_frames.append(frame)
            if len(segment_frames) == segment_length:
                base_name = os.path.splitext(os.path.basename(video_path))[0]
                output_filename = f"{base_name}_frag_{counter:03d}.mp4"
                output_path = os.path.join(output_dir, output_filename)
                save_fragment(segment_frames, output_path, fps, frame_size, fourcc)
                counter += 1
                segment_frames = []  # Очистка буфера
                
        cap.release()
        print(f"Обработано видео: {video_path}")
    
    print(f"Сегментов сохранено (man_doing_something): {counter}")

# Функция для обработки видео из базовой директории с подкатегориями
def process_videos_angle_from_base(input_base, output_base, segment_length=8, do_augmentation=True):
    """
    Входная директория (input_base) содержит поддиректории с видео (например, true, false, man_doing_something).
    Для каждой поддиректории видео обрабатываются соответствующей функцией.
    Результаты сохраняются в соответствующих папках внутри output_base.
    """
    # Список категорий
    categories = ['true', 'false', 'man_doing_something']
    # categories = [  'man_doing_something']

    
    # Создаем папки для каждой категории в output_base
    for category in categories:
        os.makedirs(os.path.join(output_base, category), exist_ok=True)
    
    for cat in categories:
        cat_input_dir = os.path.join(input_base, cat)    
        cat_output_dir = os.path.join(output_base, cat)    
        print(f"\nОбработка категории: {cat}")
        if cat == 'man_doing_something':
            process_videos_split_frames(cat_input_dir, cat_output_dir, segment_length)
        else:
            process_videos_angle_subdir(cat_input_dir, cat_output_dir, segment_length, do_augmentation)


In [3]:
def main():
    # Входная база для видео norm_v\train
    input_base = r"C:\Users\jet\Desktop\видосы\norm_v\train"
    # Выходная база – для каждой категории создается папка, куда сохраняются все сегменты
    output_base = r"C:\Users\jet\Desktop\видосы\norm_v_processed_3_0_frame\train"
    
    segment_length = 4  # Можно настроить длину сегмента в кадрах
    do_augmentation = True  # Выполняем аугментацию для категорий, где это необходимо
    
    print("Начинается обработка видео из:", input_base)
    process_videos_angle_from_base(input_base, output_base, segment_length, do_augmentation)
    
if __name__ == '__main__':
    main()


Начинается обработка видео из: C:\Users\jet\Desktop\видосы\norm_v\train

Обработка категории: true
load C:\Users\jet\.cache\rtmlib\hub\checkpoints\yolox_tiny_8xb8-300e_humanart-6f3252f9.onnx with onnxruntime backend
load C:\Users\jet\.cache\rtmlib\hub\checkpoints\rtmw-dw-l-m_simcc-cocktail14_270e-256x192_20231122.onnx with onnxruntime backend
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\train\true\true_0000.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\train\true\true_0001.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\train\true\true_0002.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\train\true\true_0003.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\train\true\true_0004.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\train\true\true_0005.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\train\true\true_0006.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\train\true\true_0007.mp4
Обработано видео: C:\Users\jet\

In [4]:
def main():
    input_base = r"C:\Users\jet\Desktop\видосы\norm_v\val"
    output_base = r"C:\Users\jet\Desktop\видосы\norm_v_processed_3_0_frame\val"
    segment_length = 4  
    do_augmentation = False  
    
    print("Начинается обработка видео из:", input_base)
    process_videos_angle_from_base(input_base, output_base, segment_length, do_augmentation)
    
if __name__ == '__main__':
    main()

Начинается обработка видео из: C:\Users\jet\Desktop\видосы\norm_v\val

Обработка категории: true
load C:\Users\jet\.cache\rtmlib\hub\checkpoints\yolox_tiny_8xb8-300e_humanart-6f3252f9.onnx with onnxruntime backend
load C:\Users\jet\.cache\rtmlib\hub\checkpoints\rtmw-dw-l-m_simcc-cocktail14_270e-256x192_20231122.onnx with onnxruntime backend
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\val\true\true_0000.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\val\true\true_0001.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\val\true\true_0002.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\val\true\true_0003.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\val\true\true_0004.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\val\true\true_0005.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\val\true\true_0006.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\val\true\true_0007.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\nor

In [5]:
def main():
    # Входная база для видео norm_v\train
    input_base = r"C:\Users\jet\Desktop\видосы\norm_v\train"
    # Выходная база – для каждой категории создается папка, куда сохраняются все сегменты
    output_base = r"C:\Users\jet\Desktop\видосы\8_0_frame\train"
    
    segment_length = 8  # Можно настроить длину сегмента в кадрах
    do_augmentation = True  # Выполняем аугментацию для категорий, где это необходимо
    
    print("Начинается обработка видео из:", input_base)
    process_videos_angle_from_base(input_base, output_base, segment_length, do_augmentation)
    
if __name__ == '__main__':
    main()


Начинается обработка видео из: C:\Users\jet\Desktop\видосы\norm_v\train

Обработка категории: true
load C:\Users\jet\.cache\rtmlib\hub\checkpoints\yolox_tiny_8xb8-300e_humanart-6f3252f9.onnx with onnxruntime backend
load C:\Users\jet\.cache\rtmlib\hub\checkpoints\rtmw-dw-l-m_simcc-cocktail14_270e-256x192_20231122.onnx with onnxruntime backend
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\train\true\true_0000.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\train\true\true_0001.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\train\true\true_0002.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\train\true\true_0003.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\train\true\true_0004.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\train\true\true_0005.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\train\true\true_0006.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\train\true\true_0007.mp4
Обработано видео: C:\Users\jet\

In [6]:
def main():
    input_base = r"C:\Users\jet\Desktop\видосы\norm_v\val"
    output_base = r"C:\Users\jet\Desktop\видосы\8_0_frame\val"
    segment_length = 8  
    do_augmentation = False  
    
    print("Начинается обработка видео из:", input_base)
    process_videos_angle_from_base(input_base, output_base, segment_length, do_augmentation)
    
if __name__ == '__main__':
    main()

Начинается обработка видео из: C:\Users\jet\Desktop\видосы\norm_v\val

Обработка категории: true
load C:\Users\jet\.cache\rtmlib\hub\checkpoints\yolox_tiny_8xb8-300e_humanart-6f3252f9.onnx with onnxruntime backend
load C:\Users\jet\.cache\rtmlib\hub\checkpoints\rtmw-dw-l-m_simcc-cocktail14_270e-256x192_20231122.onnx with onnxruntime backend
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\val\true\true_0000.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\val\true\true_0001.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\val\true\true_0002.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\val\true\true_0003.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\val\true\true_0004.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\val\true\true_0005.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\val\true\true_0006.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\norm_v\val\true\true_0007.mp4
Обработано видео: C:\Users\jet\Desktop\видосы\nor